In [ ]:
#import sys
#!{sys.executable} -m pip install imageio
#sys.executable

  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)


In [2]:
#%matplotlib widgets

In [1]:
import main as m
import pandas as pd



dataset = pd.read_csv('spx_vhs.csv')
data = dataset.values

state_arr = m.build_state_array(
    data=data,
    fss_mode='beta_vhs',
    mode='3ma60to12'
)

prob_mat = m.build_transition_matrix(
    trans_length=1,
    arr_states=state_arr
)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Size of your 2D heatmap grid
n_rows, n_cols = 60, 1
heatmaps = [[
    m.build_transition_matrix(
    trans_length=_,
    arr_states=state_arr,
    #col_filter='evens',
    #dir_norm='row'
)
 for __ in range(n_cols)] for _ in range(n_rows)]

# Track position
pos = {'row': 0, 'col': 0}

# Output widget for the plot
output = widgets.Output()

def plot_current_heatmap():
    with output:
        clear_output(wait=True)
        r, c = pos['row'], pos['col']
        plt.imshow(heatmaps[r][c], cmap='viridis')
        plt.title(f"Heatmap at [{r+1}, {c+1}]")
        plt.colorbar()
        plt.show()

def move_up(b):
    if pos['row'] > 0:
        pos['row'] -= 1
        plot_current_heatmap()

def move_down(b):
    if pos['row'] < n_rows - 1:
        pos['row'] += 1
        plot_current_heatmap()

def move_left(b):
    if pos['col'] > 0:
        pos['col'] -= 1
        plot_current_heatmap()

def move_right(b):
    if pos['col'] < n_cols - 1:
        pos['col'] += 1
        plot_current_heatmap()

# Create navigation buttons
up_btn = widgets.Button(description='↑ Up', layout=widgets.Layout(width='80px'))
down_btn = widgets.Button(description='↓ Down', layout=widgets.Layout(width='80px'))
left_btn = widgets.Button(description='← Left', layout=widgets.Layout(width='80px'))
right_btn = widgets.Button(description='→ Right', layout=widgets.Layout(width='80px'))

# Link buttons to functions
up_btn.on_click(move_up)
down_btn.on_click(move_down)
left_btn.on_click(move_left)
right_btn.on_click(move_right)

# Layout
controls = widgets.HBox([
    widgets.VBox([up_btn, down_btn]),
    widgets.VBox([left_btn, right_btn])
])

# Display everything
plot_current_heatmap()
display(controls, output)


Output()

In [19]:
print(heatmaps[44][0][5,:])

[0.24304539 0.03367496 0.13128355 0.01317716 0.05075647 0.00488043
 0.01073694 0.00195217 0.10785749 0.02244998 0.34114202 0.03904344]


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import imageio.v2 as imageio
import io

conf_matrices = heatmaps  # list of same‑shape 2D arrays, e.g. all (27,27)

images = []
for i, cm in enumerate(conf_matrices):
    cm = np.squeeze(cm)
    if cm.ndim != 2:
        raise ValueError(f"Expected 2D, got {cm.shape}")

    # fixed size + DPI
    fig, ax = plt.subplots(figsize=(5,5), dpi=100)
    cax = ax.matshow(cm, cmap='Blues')
    fig.colorbar(cax, ax=ax)

    ax.set_title(f'trans_length: {i}')
    ax.set_xlabel('Next State')
    ax.set_ylabel('crnt State')

    buf = io.BytesIO()
    # no bbox_inches – this keeps the canvas exactly 500×500 pixels
    plt.savefig(buf, format='png', dpi=100)
    buf.seek(0)
    images.append(imageio.imread(buf))
    plt.close(fig)

# now all images[i].shape == (500, 500, 3)
imageio.mimsave('confusion_matrices.gif', images, duration=0.1, loop=0)
